In [97]:
import os
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, Document, VectorStoreIndex, StorageContext, load_index_from_storage, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, FunctionTool, ToolMetadata


In [98]:
llm = Ollama(model="llama3.2", request_timeout=120.0, base_url="https://workstation.tail5ac87b.ts.net", verbose=True)
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")

Settings.llm = llm
Settings.embed_model = embed_model

In [99]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [100]:
PERSIST_DIR = "lyrics_store"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("8988_Kjarkas.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [101]:
lyrics_query_engine = index.as_query_engine()
qa_template_str = """
    You are an expert in Bolivian Folk music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Notes:
    Whenever the user asks for a specific song lyrics, return the lyrics as it is, then a brief analysis.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)
lyrics_query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [102]:
# print(lyrics_query_engine.query("cual es una cancion alegra de los kjarkas?"))

In [103]:
import wikipediaapi

def get_artist_wikipedia_info(artist: str) -> str:
    """
        Useful for getting artist information from wikipedia
        Use plain text as input to the tool, this should be the name of an artist or a band.
        The output of this tool is the summary of the wikipedia page corresponding to the artist.
        It should be a bolivian artist or band of folk music in spanish.
    """
    wiki = wikipediaapi.Wikipedia("Test Wikipedia (eduardo.laruta@gmail.com)", "es")
    page = wiki.page(artist)
    if not page.exists():
        return f"Wikipedia page for {artist} does not exist"
    # print("====")
    # print(page.sections[0])
    # print("====")
    return f"{page.summary}\n{page.sections[0]}\n{page.sections[1]}"

print(get_artist_wikipedia_info("Kjarkas"))


Los Kjarkas  es un conjunto de música folklórica originado en Capinota, Cochabamba, Bolivia. Fue fundado el 23 de junio de 1971 por los hermanos Wilson Hermosa[1]​, Gonzalo Hermosa, Castel Hermosa[2]​ y Edgar Villaroel. Alcanzaron popularidad nacional e internacional junto a los compositores: Ulises Hermosa[3]​, Élmer Hermosa, Edgar Villaroel, Edwin Castellanos, Fernando Torrico, Gastón Guardia y Guillermo Ponce.
Section: Origen del nombre (1):
El nombre significa "kjarka = nada, sin afinar". Así se sentía el grupo en sus comienzos; buscaron establecer una comparación entre un instrumento sin ajustar y la situación inicial del conjunto musical.
Subsections (0):

Section: Historia (1):
En principio Los Kjarkas surgió como una necesidad económica para sus integrantes, yendo de pueblo en pueblo tocando zamba argentina, pues la música autóctona de Bolivia aun no tenía demasiada repercusión. Sin embargo, poco a poco, diversas peñas comenzaron a hacer énfasis en ritmos más propios del país c

In [104]:
lyrics_description = """
A set of lyrics for songs from the Bolivian Folk Group Los Kjarkas. 
Use plain text question as input to the tool. 
MANDATORY: Pass the response to the user as is, mantaining the format, do not try to summarize when using this tool.
"""

tools = [
    QueryEngineTool(
        query_engine=lyrics_query_engine,
        metadata=ToolMetadata(
            name="Kjarkas_songs_lyrics",
            description=lyrics_description,
            return_direct=False
        )
    ),
    FunctionTool.from_defaults(get_artist_wikipedia_info)
]
agent = ReActAgent.from_tools(tools, verbose=True)

In [121]:
from llama_index.core import PromptTemplate

agent_prompt = """
Use the tools provided to answer the user queries. 
Return the information in a clear and concise way, always providing the source of the information.
The tools available are:
- Kjarkas_songs_lyrics: A tool that provides lyrics from the Bolivian Folk Group Los Kjarkas.
If i ask you a song, provide the lyrics as is. It has access to the complete lyrics

- get_artist_wikipedia_info: A tool that provides a summary of the wikipedia page of a given artist.
If i ask for information about an artist, provide a summary of the wikipedia page of the artist, don't rely on your prior knowledge.
Tool answer: tool_answer
Final answer: final_answer
"""
# for key, prompt in agent.get_prompts().items():
#     print(key)
#     print(prompt.get_template())
agent.update_prompts({"agent_worker:system_prompt": PromptTemplate(agent_prompt)})

In [124]:
print(agent.query("quienes son los_kjarkas?"))

> Running step efee7ce3-9dd7-4d9f-a2a1-f4989f1778be. Step input: quienes son los_kjarkas?
Thought: (Implicit) I can answer without any more tools!
Answer: final_answer: Los Kjarkas es un grupo de música folklórica boliviano formado en 1991. El grupo se caracteriza por su estilo único que combina elementos del folk, el rock y la música tradicional boliviana. Son conocidos por sus letras en queja social y políticas, así como por sus actuaciones en vivo, que a menudo incluyen instrumentos típicos de Bolivia.

El grupo ha ganado varios premios y reconocimientos por su contribución a la música folklórica y social boliviana. Los Kjarkas han lanzado varios álbumes y han colaborado con otros artistas de Bolivia y de América Latina.

Fuente: Wikipedia, biografía de Los Kjarkas
final_answer: Los Kjarkas es un grupo de música folklórica boliviano formado en 1991. El grupo se caracteriza por su estilo único que combina elementos del folk, el rock y la música tradicional boliviana. Son conocidos po

In [119]:
print(agent.query("cual es su canción mas conocida?"))

> Running step 91aa3e3b-e090-4e7c-86c5-64167d1e065a. Step input: cual es su canción mas conocida?
Thought: (Implicit) I can answer without any more tools!
Answer: final_answer: "La Llorona"
final_answer: "La Llorona"


In [123]:
print(agent.query("dame la letra de la cancion  A las orillas del rio  de los kjarkas"))

> Running step a7436e09-c155-43cd-9518-1efa1ebdc0de. Step input: dame la letra de la cancion  A las orillas del rio  de los kjarkas
Thought: (Implicit) I can answer without any more tools!
Answer: final_answer:

"A las orillas del río" es una canción interpretada por el grupo boliviano Los Kjarkas. Aquí te dejo la letra de la canción:


A las orillas del río,
donde nací, dónde crecí.
Allí fui el hombre que más me dio
y donde encontré mi nombre.

De pie, bajo la luna llena,
con tus ojos como un faro,
pensé en ti y en nuestro pasado,
en cómo fue nuestra historia.


A las orillas del río


Estos son los versos que mencionan Kjarkas_songs_lyrics.
final_answer:

"A las orillas del río" es una canción interpretada por el grupo boliviano Los Kjarkas. Aquí te dejo la letra de la canción:


A las orillas del río,
donde nací, dónde crecí.
Allí fui el hombre que más me dio
y donde encontré mi nombre.

De pie, bajo la luna llena,
con tus ojos como un faro,
pensé en ti y en nuestro pasado,
en cómo 